In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import scipy.special as sp

In [2]:
%matplotlib inline

In [3]:
# Определение класса нейронной сети. C ф-циями инициализации, терировки, опросом(после работы трех слоев)
class neuralNetwork:

    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        self.innodes = inputnodes   # количество узлов входного слоя
        self.hnodes = hiddennodes   # количество узлов скрытого слоя 
        self.onodes = outputnodes   # количество узлов выходного слоя 
        
        self.lr = learningrate  # коэффициент обучения
        self.activation_function = lambda x : sp.expit(x) # функция активации(сигмоида)

        # Матрица связей коэф-ов wih (входной -> скрытый) и 
        # Матрица связей коэф-ов who (скрытый -> выходной)
        # Весовые коэффициенты связей между узлом i и узлом j следующего слоя обозначим как w_i_j
        # i     j     k
        # w11   w21   w31
        # w12   w22   w32
        # w13   w23   w33
        # Ниже мы задали:
        # матрица W весовых коэф. размерностью [hnodes X innodes]
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.innodes)) #центр норм-го распр-я установлен в нуле(0.0)
        # матрица W весовых коэф. размерностью [onodes X hnodes]
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))


    # Тренировка нейронной сети:
    # Уточнение весовых коэффициентов в процессе обработки предоставленных для обучения сети терн.примеров
    def train(self, inputs_list, targets_list):

        #targets = np.array(targets_list, ndmin=2).T
        # Сама непосредственно тренировка включает две фазы:
        # 1. Расчет выходного сигнала(ф-ция query)
        # 2. Обратное распространение ошибок, информурующее нас о том, каковы должны быть поправки к весам
        
        # Преобразовать список входных значений в двухмерный массив
        inputs = np.array(inputs_list, ndmin=2).T   # транспонируем т.к. на вход строка , а нужен столбец
        targets = np.array(targets_list, ndmin=2).T # ndim - кол-ство осей в массиве. Т.е. 2 - двухмерный массив

        #Код аналогичный в query --------------------------------
        # Расчитать входящие сигналы для скрытого слоя
        hidden_inputs = np.dot(self.wih, inputs)
        # Расчитать выходящие сигналы из скрытого слоя 
        hidden_outputs = self.activation_function(hidden_inputs) 

        # Расчитать входящие сигналы для выходного слоя
        final_inputs = np.dot(self.who, hidden_outputs)
        # Расчитать входящие сигналы для выходного слоя
        final_outputs = self.activation_function(final_inputs)
        #--------------------------------------------------------
        

        # Ошибка = целевое значение(target) - фактическое значение
        output_errors = targets - final_outputs

        # Ошибки скрытого слоя - это ошибки output_errors, распределенные прапорционально
        # весовым коэффициентам связей и рекомбинированные на скрытых узлах
        hidden_errors = np.dot(self.who.T, output_errors) 

        # Обновить весовые коэфф. для связей между скрытым и выходным слоями
        self.who += self.lr * np.dot(output_errors * final_outputs * (1.0 - final_outputs), np.transpose(hidden_outputs))

        # Обновить весовые коэфф. для связей между входным и скрытым слоями
        self.wih += self.lr * np.dot(hidden_errors * hidden_outputs * (1.0 - hidden_outputs), np.transpose(inputs))


    # Опрос нейронной сети:
    # Получение значений сигналов с выходных узлов после предоствления значений входящих сигналов
    # (т.е. в качестве аргумента входные данные НС, возвращает выходные данные НС)
    def query(self, inputs_list):
        # преобразовать список входных значений в двумерный массив
        # и выполнить транспонирование, для умножения матрицы W(к-ство строк) на матрицу I(к-ство столбцов)  
        inputs_list = np.array(inputs_list, ndmin=2).T

        # расчитать входящие сигналы в скрытый слой
        hidden_inputs = np.dot(self.wih, inputs_list)
        # расчитать исходящие сигналы из скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)

        # расчитать входящие сигналы в выходной слой
        final_inputs = np.dot(self.who, hidden_outputs)
        # расчитать исходящие сигналы из выходного слоя
        final_outputs = self.activation_function(final_inputs)

        return final_outputs


In [4]:

input_nodes = 784   #количество пикселей из которых состоит изображение рукописной цифры
hidden_nodes = 100  #100? не имею строгого научного обоснования 
output_nodes = 10

learning_rate = 0.3
#создаем класс нейросети
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

training_data_file = open("./mnist_dataset/mnist_train_100.csv", "r")
training_data_list = training_data_file.readlines()
training_data_file.close()

#тренировка сети

#перебрать все записи в тренировочном наборе данных
for record in training_data_list:
    all_values = record.split(",")  
    inputs = (np.asarray(all_values[1:], dtype=np.float64) / 255.0 * 0.99) + 0.01
    targets = np.zeros(output_nodes) + 0.01
    targets[int(all_values[0])] = 0.99
    n.train(inputs, targets)

In [5]:
test_data_file = open("./mnist_dataset/mnist_test_10.csv", "r")
test_data_list = test_data_file.readlines()
test_data_file.close()

all_values_test = test_data_list[0].split(",")
print(all_values_test)


['7', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',

In [6]:
n.query(np.asarray(all_values_test[1:], dtype=np.float64) / 255.0 * 0.99) + 0.01

array([[0.08495774],
       [0.0499208 ],
       [0.06567209],
       [0.10858558],
       [0.0515901 ],
       [0.04702168],
       [0.02392128],
       [0.92204486],
       [0.07282083],
       [0.04522078]])

In [7]:
# Цифру 7 наша сеть распознала корректно, посмотрим как она справится с остальной частью данных для тестирования

# Журнал оценок работы сети
scorecard = []

# Переберем все записи в тестовом наборе данных
for record in test_data_list:
    all_values = record.split(",")   
    correct_label = int(all_values[0])  # Правильный ответ - первое значение
    print("Маркерное значение(истинное значние): ", correct_label)
    inputs = (np.asarray(all_values[1:], dtype=np.float64) / 255.0 * 0.99) + 0.01 #масштабируем и смещаем входные значения
    outputs = n.query(inputs)
    label = np.argmax(outputs) #индекс наибольшего значения является маркерным значением   
    print(label, " ответ сети")    
    # Присоединить оценку ответа сети к концу списка
    if (label == correct_label):
        # Если сеть правильно определила значение, присоединить к списку 1
        scorecard.append(1)
    else:
        scorecard.append(0)

print(scorecard)

Маркерное значение(истинное значние):  7
7  ответ сети
Маркерное значение(истинное значние):  2
0  ответ сети
Маркерное значение(истинное значние):  1
1  ответ сети
Маркерное значение(истинное значние):  0
0  ответ сети
Маркерное значение(истинное значние):  4
4  ответ сети
Маркерное значение(истинное значние):  1
1  ответ сети
Маркерное значение(истинное значние):  4
4  ответ сети
Маркерное значение(истинное значние):  9
3  ответ сети
Маркерное значение(истинное значние):  5
0  ответ сети
Маркерное значение(истинное значние):  9
7  ответ сети
[1, 0, 1, 1, 1, 1, 1, 0, 0, 0]


In [8]:
# Не самый хороший результат, так как из 10 тестовых значений правильно распознаных - 6
# Выведем долю правильно распознаных ответов
scorecard_array = np.asarray(scorecard)
print("Эффективность НС = ", scorecard_array.sum() / scorecard_array.size) 

Эффективность НС =  0.6
